In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import pandas as pd

## Dataset 1:

In [3]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the ESPN rankings page
url = "https://www.espn.com/college-football/rankings/_/poll/1/week/1/year/2024/seasontype/3"
browser.get(url)

time.sleep(random.uniform(2, 5))

try:
    table = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
    rows = table.find_elements(By.TAG_NAME, "tr")

    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = [col.text.strip() for col in cols]
        if cols_text:  # Avoid empty rows
            data.append(cols_text)

except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

rank_df = pd.DataFrame(data, columns=headers)
rank_df.head()

,RK,TEAM,REC,PTS,TREND,LAST WEEK,NEXT WEEK
0,1,Ohio State\n(56),14-2,1400,-,vsNotre DameW34-23,-
1,2,Notre Dame,14-2,1342,-,vsOhio StateL34-23,-
2,3,Oregon,13-1,1255,-,vsOhio StateL41-21,-
3,4,Texas,13-3,1211,-,vsOhio StateL28-14,-
4,5,Penn State,13-3,1203,-,vsNotre DameL27-24,-


In [4]:
rank_df = rank_df[["RK", "TEAM", "REC"]] 
rank_df.head()

,RK,TEAM,REC
0,1,Ohio State\n(56),14-2
1,2,Notre Dame,14-2
2,3,Oregon,13-1
3,4,Texas,13-3
4,5,Penn State,13-3


In [5]:
rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RK      25 non-null     object
 1   TEAM    25 non-null     object
 2   REC     25 non-null     object
dtypes: object(3)
memory usage: 732.0+ bytes


In [6]:
rank_df["RK"] = pd.to_numeric(rank_df["RK"])
rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RK      25 non-null     int64 
 1   TEAM    25 non-null     object
 2   REC     25 non-null     object
dtypes: int64(1), object(2)
memory usage: 732.0+ bytes


In [7]:
#Cleaning so Ohio State is in a standard format
import re
rank_df["TEAM"] = rank_df["TEAM"].str.replace(r"[^a-zA-Z\s]", "", regex=True).str.strip()
rank_df

,RK,TEAM,REC
0,1,Ohio State,14-2
1,2,Notre Dame,14-2
2,3,Oregon,13-1
3,4,Texas,13-3
4,5,Penn State,13-3
5,6,Georgia,11-3
6,7,Arizona State,11-3
7,8,Boise State,12-2
8,9,Tennessee,10-3
9,10,Indiana,11-2


In [8]:
rank_df = rank_df.rename(columns = {
    'RK' : 'RANK',
    'REC' : 'RECORD'
})
rank_df.head()

,RANK,TEAM,RECORD
0,1,Ohio State,14-2
1,2,Notre Dame,14-2
2,3,Oregon,13-1
3,4,Texas,13-3
4,5,Penn State,13-3


## Dataset 2

In [10]:
service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Defensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=D&cat=T&yr=2024&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)


except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    defense_df = pd.DataFrame(data, columns=headers)
    defense_df['Team'] = defense_df['Team'].str.replace('St.', 'State', regex=False)
else:
    Defense_df = pd.DataFrame(data) 

defense_df

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,Ohio State,16,206,12.9,1402,87.6,2306,144.1,3708,231.8
1,Indiana,13,203,15.6,1043,80.2,2039,156.8,3082,237.1
2,Texas,16,245,15.3,1758,109.9,2475,154.7,4233,264.6
3,N. Illinois,13,241,18.5,1459,112.2,2043,157.2,3502,269.4
4,Minnesota,13,220,16.9,1425,109.6,2103,161.8,3528,271.4
...,...,...,...,...,...,...,...,...,...,...
129,Utah State,12,453,37.8,2574,214.5,2891,240.9,5465,455.4
130,Oklahoma State,12,427,35.6,2580,215.0,3239,269.9,5819,484.9
131,New Mexico,12,456,38.0,2555,212.9,3298,274.8,5853,487.8
132,Tulsa,12,510,42.5,2169,180.8,3691,307.6,5860,488.3


In [11]:
defense_df = defense_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
defense_df.head()

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,Ohio State,12.9,87.6,144.1,231.8
1,Indiana,15.6,80.2,156.8,237.1
2,Texas,15.3,109.9,154.7,264.6
3,N. Illinois,18.5,112.2,157.2,269.4
4,Minnesota,16.9,109.6,161.8,271.4


In [12]:
defense_df = defense_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS ALLOWED/GAME",
    "RYds/G": "RUSH YDS ALLOWED/GAME",
    "PYds/G": "PASS YDS ALLOWED/GAME",
    "Yds/G": "YARDS ALLOWED/GAME"
})
defense_df.head()

,TEAM,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME
0,Ohio State,12.9,87.6,144.1,231.8
1,Indiana,15.6,80.2,156.8,237.1
2,Texas,15.3,109.9,154.7,264.6
3,N. Illinois,18.5,112.2,157.2,269.4
4,Minnesota,16.9,109.6,161.8,271.4


In [13]:
#Cleaning the data so the team names match with ranks_df
defense_df['TEAM'] = defense_df['TEAM'].str.replace('Mississippi', 'Ole Miss', regex=False)
defense_df['TEAM'] = defense_df['TEAM'].str.replace('S. Carolina', 'South Carolina', regex=False)
defense_df.head()

,TEAM,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME
0,Ohio State,12.9,87.6,144.1,231.8
1,Indiana,15.6,80.2,156.8,237.1
2,Texas,15.3,109.9,154.7,264.6
3,N. Illinois,18.5,112.2,157.2,269.4
4,Minnesota,16.9,109.6,161.8,271.4


In [14]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   TEAM                   134 non-null    object
 1   POINTS ALLOWED/GAME    134 non-null    object
 2   RUSH YDS ALLOWED/GAME  134 non-null    object
 3   PASS YDS ALLOWED/GAME  134 non-null    object
 4   YARDS ALLOWED/GAME     134 non-null    object
dtypes: object(5)
memory usage: 5.4+ KB


In [15]:
defense_df["POINTS ALLOWED/GAME"] = pd.to_numeric(defense_df["POINTS ALLOWED/GAME"])
defense_df["RUSH YDS ALLOWED/GAME"] = pd.to_numeric(defense_df["RUSH YDS ALLOWED/GAME"])
defense_df["PASS YDS ALLOWED/GAME"] = pd.to_numeric(defense_df["PASS YDS ALLOWED/GAME"])
defense_df["YARDS ALLOWED/GAME"] = pd.to_numeric(defense_df["YARDS ALLOWED/GAME"])
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TEAM                   134 non-null    object 
 1   POINTS ALLOWED/GAME    134 non-null    float64
 2   RUSH YDS ALLOWED/GAME  134 non-null    float64
 3   PASS YDS ALLOWED/GAME  134 non-null    float64
 4   YARDS ALLOWED/GAME     134 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.4+ KB


## Dataset 3

In [17]:
service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Offensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=O&cat=T&yr=2024&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)


except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    offense_df = pd.DataFrame(data, columns=headers)
    offense_df['Team'] = offense_df['Team'].str.replace('St.', 'State', regex=False)
else:
    offense_df = pd.DataFrame(data) 

offense_df                    

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,Miami,13,571,43.9,2456,188.9,4378,336.8,6834,525.7
1,Mississippi,13,502,38.6,2286,175.8,4395,338.1,6681,513.9
2,N. Texas,13,436,33.5,2088,160.6,4182,321.7,6270,482.3
3,New Mexico,12,402,33.5,3044,253.7,2743,228.6,5787,482.2
4,Texas State,13,475,36.5,2712,208.6,3401,261.6,6113,470.2
...,...,...,...,...,...,...,...,...,...,...
129,Houston,12,168,14.0,1618,134.8,1615,134.6,3233,269.4
130,S. Miss,12,183,15.2,1307,108.9,1799,149.9,3106,258.8
131,Florida State,12,185,15.4,1079,89.9,1901,158.4,2980,248.3
132,Kennesaw State,12,198,16.5,1375,114.6,1515,126.2,2890,240.8


In [18]:
offense_df = offense_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
offense_df

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,Miami,43.9,188.9,336.8,525.7
1,Mississippi,38.6,175.8,338.1,513.9
2,N. Texas,33.5,160.6,321.7,482.3
3,New Mexico,33.5,253.7,228.6,482.2
4,Texas State,36.5,208.6,261.6,470.2
...,...,...,...,...,...
129,Houston,14.0,134.8,134.6,269.4
130,S. Miss,15.2,108.9,149.9,258.8
131,Florida State,15.4,89.9,158.4,248.3
132,Kennesaw State,16.5,114.6,126.2,240.8


In [19]:
offense_df = offense_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS/GAME",
    "RYds/G": "RUSH YDS/GAME",
    "PYds/G": "PASS YDS/GAME",
    "Yds/G": "YARDS/GAME"
})
offense_df.head(15)

,TEAM,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME
0,Miami,43.9,188.9,336.8,525.7
1,Mississippi,38.6,175.8,338.1,513.9
2,N. Texas,33.5,160.6,321.7,482.3
3,New Mexico,33.5,253.7,228.6,482.2
4,Texas State,36.5,208.6,261.6,470.2
5,Boise State,37.3,240.4,220.3,460.6
6,Utah State,31.9,199.2,261.2,460.5
7,Texas Tech,37.6,166.1,283.7,449.8
8,Syracuse,34.1,97.6,352.0,449.6
9,Arkansas,30.9,184.8,256.3,441.1


In [20]:
#Cleaning the data so the team names match with ranks_df
offense_df['TEAM'] = offense_df['TEAM'].str.replace('Mississippi', 'Ole Miss', regex=False)
offense_df['TEAM'] = offense_df['TEAM'].str.replace('S. Carolina', 'South Carolina', regex=False)
offense_df.head(15)

,TEAM,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME
0,Miami,43.9,188.9,336.8,525.7
1,Ole Miss,38.6,175.8,338.1,513.9
2,N. Texas,33.5,160.6,321.7,482.3
3,New Mexico,33.5,253.7,228.6,482.2
4,Texas State,36.5,208.6,261.6,470.2
5,Boise State,37.3,240.4,220.3,460.6
6,Utah State,31.9,199.2,261.2,460.5
7,Texas Tech,37.6,166.1,283.7,449.8
8,Syracuse,34.1,97.6,352.0,449.6
9,Arkansas,30.9,184.8,256.3,441.1


In [21]:
offense_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TEAM           134 non-null    object
 1   POINTS/GAME    134 non-null    object
 2   RUSH YDS/GAME  134 non-null    object
 3   PASS YDS/GAME  134 non-null    object
 4   YARDS/GAME     134 non-null    object
dtypes: object(5)
memory usage: 5.4+ KB


In [22]:
offense_df["POINTS/GAME"] = pd.to_numeric(offense_df["POINTS/GAME"])
offense_df["RUSH YDS/GAME"] = pd.to_numeric(offense_df["RUSH YDS/GAME"])
offense_df["PASS YDS/GAME"] = pd.to_numeric(offense_df["PASS YDS/GAME"])
offense_df["YARDS/GAME"] = pd.to_numeric(offense_df["YARDS/GAME"])
offense_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TEAM           134 non-null    object 
 1   POINTS/GAME    134 non-null    float64
 2   RUSH YDS/GAME  134 non-null    float64
 3   PASS YDS/GAME  134 non-null    float64
 4   YARDS/GAME     134 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.4+ KB


## Merged DF

In [24]:
# Merge rank_df with defense_df on 'TEAM'
merged_df = rank_df.merge(defense_df, on="TEAM", how="left")

# Merge the result with offense_df on 'TEAM'
merged_df = merged_df.merge(offense_df, on="TEAM", how="left")
merged_df['Season'] = 2024
# Display the final merged DataFrame
merged_df


,RANK,TEAM,RECORD,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME,Season
0,1,Ohio State,14-2,12.9,87.6,144.1,231.8,35.7,166.4,256.4,422.8,2024
1,2,Notre Dame,14-2,15.5,138.1,152.8,290.9,36.1,200.9,189.5,390.4,2024
2,3,Oregon,13-1,19.4,130.1,171.1,301.2,34.9,157.9,268.5,426.4,2024
3,4,Texas,13-3,15.3,109.9,154.7,264.6,33.0,158.8,264.9,423.7,2024
4,5,Penn State,13-3,16.5,101.9,173.6,275.4,33.1,202.3,219.0,421.3,2024
5,6,Georgia,11-3,20.6,130.1,183.4,313.5,31.5,124.4,267.6,392.1,2024
6,7,Arizona State,11-3,22.6,113.1,212.6,325.7,32.9,200.2,219.1,419.4,2024
7,8,Boise State,12-2,22.6,122.3,214.4,336.6,37.3,240.4,220.3,460.6,2024
8,9,Tennessee,10-3,16.1,103.9,175.5,279.4,35.7,226.5,209.5,435.9,2024
9,10,Indiana,11-2,15.6,80.2,156.8,237.1,41.3,165.1,249.2,414.2,2024


In [25]:
merged_df.to_csv("CFB_2024.csv", index=False)

# 2023 season:

In [27]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the ESPN rankings page
url = "https://www.espn.com/college-football/rankings/_/poll/1/week/1/year/2023/seasontype/3"
browser.get(url)

time.sleep(random.uniform(2, 5))

try:
    table = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
    rows = table.find_elements(By.TAG_NAME, "tr")

    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = [col.text.strip() for col in cols]
        if cols_text:  # Avoid empty rows
            data.append(cols_text)

except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

rank_df2 = pd.DataFrame(data, columns=headers)
rank_df2.head()

,RK,TEAM,REC,PTS,TREND,LAST WEEK,NEXT WEEK
0,1,Michigan\n(61),15-0,1525,-,vs #2WashingtonW34-13,-
1,2,Washington,14-1,1459,-,vs #1MichiganL34-13,-
2,3,Texas,12-2,1356,-,vs #2WashingtonL37-31,-
3,4,Georgia,13-1,1328,-,vs #6Florida StateW63-3,-
4,5,Alabama,12-2,1321,-,vs #1MichiganL27-20,-


In [28]:
rank_df2 = rank_df2[["RK", "TEAM", "REC"]] 
rank_df2

,RK,TEAM,REC
0,1,Michigan\n(61),15-0
1,2,Washington,14-1
2,3,Texas,12-2
3,4,Georgia,13-1
4,5,Alabama,12-2
5,6,Florida State,13-1
6,6,Oregon,12-2
7,8,Missouri,11-2
8,9,Ole Miss,11-2
9,10,Ohio State,11-2


In [29]:
rank_df2["RK"] = pd.to_numeric(rank_df2["RK"])
rank_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RK      25 non-null     int64 
 1   TEAM    25 non-null     object
 2   REC     25 non-null     object
dtypes: int64(1), object(2)
memory usage: 732.0+ bytes


In [30]:
import re
rank_df2["TEAM"] = rank_df2["TEAM"].str.replace(r"[^a-zA-Z\s]", "", regex=True).str.strip()
rank_df2

,RK,TEAM,REC
0,1,Michigan,15-0
1,2,Washington,14-1
2,3,Texas,12-2
3,4,Georgia,13-1
4,5,Alabama,12-2
5,6,Florida State,13-1
6,6,Oregon,12-2
7,8,Missouri,11-2
8,9,Ole Miss,11-2
9,10,Ohio State,11-2


In [31]:
rank_df2 = rank_df2.rename(columns = {
    'RK' : 'RANK',
    'REC' : 'RECORD'
})
rank_df2

,RANK,TEAM,RECORD
0,1,Michigan,15-0
1,2,Washington,14-1
2,3,Texas,12-2
3,4,Georgia,13-1
4,5,Alabama,12-2
5,6,Florida State,13-1
6,6,Oregon,12-2
7,8,Missouri,11-2
8,9,Ole Miss,11-2
9,10,Ohio State,11-2


# 2023 defense

In [33]:
 service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Defensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=D&cat=T&yr=2023&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)

except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    defense2_df = pd.DataFrame(data, columns=headers)
    defense2_df['Team'] = defense2_df['Team'].str.replace('St.', 'State', regex=False)
else:
    Defense2_df = pd.DataFrame(data) 

defense2_df

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,Penn State,13,175,13.5,982,75.5,1903,146.4,2885,221.9
1,Michigan,15,156,10.4,1350,90.0,2057,137.1,3407,227.1
2,Ohio State,13,146,11.2,1553,119.5,1725,132.7,3278,252.2
3,Ohio,13,206,15.8,1185,91.2,2176,167.4,3361,258.5
4,Notre Dame,13,207,15.9,1541,118.5,1824,140.3,3365,258.8
...,...,...,...,...,...,...,...,...,...,...
128,Vanderbilt,12,434,36.2,2104,175.3,3224,268.7,5328,444.0
129,Memphis,12,353,29.4,2034,169.5,3306,275.5,5340,445.0
130,Stanford,12,452,37.7,1964,163.7,3402,283.5,5366,447.2
131,UAB,11,398,36.2,2365,215.0,2697,245.2,5062,460.2


In [34]:
defense2_df = defense2_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
defense2_df.head()

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,Penn State,13.5,75.5,146.4,221.9
1,Michigan,10.4,90.0,137.1,227.1
2,Ohio State,11.2,119.5,132.7,252.2
3,Ohio,15.8,91.2,167.4,258.5
4,Notre Dame,15.9,118.5,140.3,258.8


In [35]:
defense2_df = defense2_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS ALLOWED/GAME",
    "RYds/G": "RUSH YDS ALLOWED/GAME",
    "PYds/G": "PASS YDS ALLOWED/GAME",
    "Yds/G": "YARDS ALLOWED/GAME"
})
defense2_df.head()

,TEAM,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME
0,Penn State,13.5,75.5,146.4,221.9
1,Michigan,10.4,90.0,137.1,227.1
2,Ohio State,11.2,119.5,132.7,252.2
3,Ohio,15.8,91.2,167.4,258.5
4,Notre Dame,15.9,118.5,140.3,258.8


In [36]:
defense2_df["POINTS ALLOWED/GAME"] = pd.to_numeric(defense2_df["POINTS ALLOWED/GAME"])
defense2_df["RUSH YDS ALLOWED/GAME"] = pd.to_numeric(defense2_df["RUSH YDS ALLOWED/GAME"])
defense2_df["PASS YDS ALLOWED/GAME"] = pd.to_numeric(defense2_df["PASS YDS ALLOWED/GAME"])
defense2_df["YARDS ALLOWED/GAME"] = pd.to_numeric(defense2_df["YARDS ALLOWED/GAME"])
defense2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TEAM                   133 non-null    object 
 1   POINTS ALLOWED/GAME    133 non-null    float64
 2   RUSH YDS ALLOWED/GAME  133 non-null    float64
 3   PASS YDS ALLOWED/GAME  133 non-null    float64
 4   YARDS ALLOWED/GAME     133 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.3+ KB


In [37]:
#Cleaning the data so the team names match with ranks_df
defense2_df['TEAM'] = defense2_df['TEAM'].str.replace('Mississippi', 'Ole Miss', regex=False)

# 2023 Offense

In [39]:
service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Offensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=O&cat=T&yr=2023&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)


except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    offense2_df = pd.DataFrame(data, columns=headers)
    offense2_df['Team'] = offense2_df['Team'].str.replace('St.', 'State', regex=False)
else:
    offense2_df = pd.DataFrame(data) 

offense2_df                    

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,LSU,13,592,45.5,2658,204.5,4312,331.7,6970,536.2
1,Oregon,14,619,44.2,2583,184.5,4826,344.7,7409,529.2
2,Oklahoma,13,542,41.7,2368,182.2,4109,316.1,6477,498.2
3,Liberty,14,536,38.3,4106,293.3,2830,202.1,6936,495.4
4,Georgia,14,562,40.1,2676,191.1,4194,299.6,6870,490.7
...,...,...,...,...,...,...,...,...,...,...
128,E. Carolina,12,208,17.3,1360,113.3,1789,149.1,3149,262.4
129,Akron,12,195,16.2,972,81.0,2049,170.8,3021,251.8
130,Kent State,12,176,14.7,1272,106.0,1720,143.3,2992,249.3
131,E. Michigan,13,254,19.5,1457,112.1,1770,136.2,3227,248.2


In [40]:
offense2_df = offense2_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
offense2_df

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,LSU,45.5,204.5,331.7,536.2
1,Oregon,44.2,184.5,344.7,529.2
2,Oklahoma,41.7,182.2,316.1,498.2
3,Liberty,38.3,293.3,202.1,495.4
4,Georgia,40.1,191.1,299.6,490.7
...,...,...,...,...,...
128,E. Carolina,17.3,113.3,149.1,262.4
129,Akron,16.2,81.0,170.8,251.8
130,Kent State,14.7,106.0,143.3,249.3
131,E. Michigan,19.5,112.1,136.2,248.2


In [41]:
offense2_df = offense2_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS/GAME",
    "RYds/G": "RUSH YDS/GAME",
    "PYds/G": "PASS YDS/GAME",
    "Yds/G": "YARDS/GAME"
})
offense2_df.head(15)

,TEAM,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME
0,LSU,45.5,204.5,331.7,536.2
1,Oregon,44.2,184.5,344.7,529.2
2,Oklahoma,41.7,182.2,316.1,498.2
3,Liberty,38.3,293.3,202.1,495.4
4,Georgia,40.1,191.1,299.6,490.7
5,Memphis,38.9,164.7,324.1,488.8
6,N. Carolina,34.5,192.7,285.4,478.1
7,N. Texas,34.5,191.2,284.1,475.2
8,UCF,31.3,228.2,245.1,473.3
9,UAB,30.7,175.7,296.9,472.6


In [42]:
offense2_df["POINTS/GAME"] = pd.to_numeric(offense2_df["POINTS/GAME"])
offense2_df["RUSH YDS/GAME"] = pd.to_numeric(offense2_df["RUSH YDS/GAME"])
offense2_df["PASS YDS/GAME"] = pd.to_numeric(offense2_df["PASS YDS/GAME"])
offense2_df["YARDS/GAME"] = pd.to_numeric(offense2_df["YARDS/GAME"])
offense2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TEAM           133 non-null    object 
 1   POINTS/GAME    133 non-null    float64
 2   RUSH YDS/GAME  133 non-null    float64
 3   PASS YDS/GAME  133 non-null    float64
 4   YARDS/GAME     133 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.3+ KB


In [43]:
#Cleaning the data so the team names match with ranks_df
offense2_df['TEAM'] = offense2_df['TEAM'].str.replace('Mississippi', 'Ole Miss', regex=False)

In [44]:
# Merge rank_df with defense_df on 'TEAM'
merged2_df = rank_df2.merge(defense2_df, on="TEAM", how="left")

# Merge the result with offense_df on 'TEAM'
merged2_df = merged2_df.merge(offense2_df, on="TEAM", how="left")
merged2_df['Season'] = 2023
# Display the final merged DataFrame
merged2_df

,RANK,TEAM,RECORD,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME,Season
0,1,Michigan,15-0,10.4,90.0,137.1,227.1,35.9,169.1,203.5,372.5,2023
1,2,Washington,14-1,24.8,148.1,249.0,397.1,36.0,118.4,337.9,456.3,2023
2,3,Texas,12-2,18.9,82.4,238.9,321.3,35.8,188.4,278.4,466.9,2023
3,4,Georgia,13-1,15.6,113.6,160.1,273.8,40.1,191.1,299.6,490.7,2023
4,5,Alabama,12-2,19.0,124.9,173.0,297.9,34.0,172.6,194.6,367.3,2023
5,6,Florida State,13-1,19.3,152.2,158.2,310.4,34.6,150.1,237.5,387.6,2023
6,6,Oregon,12-2,16.5,102.6,196.8,299.4,44.2,184.5,344.7,529.2,2023
7,8,Missouri,11-2,20.8,122.7,194.1,316.8,32.5,172.0,251.8,423.8,2023
8,9,Ole Miss,11-2,22.5,153.1,212.9,366.0,35.1,176.5,273.2,449.6,2023
9,10,Ohio State,11-2,11.2,119.5,132.7,252.2,30.5,138.8,258.8,397.7,2023


# 2022 Season

In [46]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the ESPN rankings page
url = "https://www.espn.com/college-football/rankings/_/poll/1/week/1/year/2022/seasontype/3"
browser.get(url)

time.sleep(random.uniform(2, 5))

try:
    table = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
    rows = table.find_elements(By.TAG_NAME, "tr")

    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = [col.text.strip() for col in cols]
        if cols_text:  # Avoid empty rows
            data.append(cols_text)

except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

rank_df3 = pd.DataFrame(data, columns=headers)
rank_df3.head()

,RK,TEAM,REC,PTS,TREND,LAST WEEK,NEXT WEEK
0,1,Georgia\n(63),15-0,1575,-,vs #2TCUW65-7,-
1,2,TCU,13-2,1484,-,vs #1GeorgiaL65-7,-
2,3,Michigan,13-1,1438,-,vs #2TCUL51-45,-
3,4,Ohio State,11-2,1394,-,vs #1GeorgiaL42-41,-
4,5,Alabama,11-2,1303,-,vs #14Kansas StateW45-20,-


In [47]:
rank_df3 = rank_df3[["RK", "TEAM", "REC"]] 
rank_df3

,RK,TEAM,REC
0,1,Georgia\n(63),15-0
1,2,TCU,13-2
2,3,Michigan,13-1
3,4,Ohio State,11-2
4,5,Alabama,11-2
5,6,Tennessee,11-2
6,7,Penn State,11-2
7,8,Washington,11-2
8,9,Tulane,12-2
9,10,Utah,10-4


In [48]:
rank_df3["RK"] = pd.to_numeric(rank_df3["RK"])
rank_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RK      25 non-null     int64 
 1   TEAM    25 non-null     object
 2   REC     25 non-null     object
dtypes: int64(1), object(2)
memory usage: 732.0+ bytes


In [49]:
import re
rank_df3["TEAM"] = rank_df3["TEAM"].str.replace(r"[^a-zA-Z\s]", "", regex=True).str.strip()
rank_df3.head()

,RK,TEAM,REC
0,1,Georgia,15-0
1,2,TCU,13-2
2,3,Michigan,13-1
3,4,Ohio State,11-2
4,5,Alabama,11-2


In [50]:
rank_df3 = rank_df3.rename(columns = {
    'RK' : 'RANK',
    'REC' : 'RECORD'
})
rank_df3.head()

,RANK,TEAM,RECORD
0,1,Georgia,15-0
1,2,TCU,13-2
2,3,Michigan,13-1
3,4,Ohio State,11-2
4,5,Alabama,11-2


# 2022 Defense

In [52]:
service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Defensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=D&cat=T&yr=2022&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)

except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    defense3_df = pd.DataFrame(data, columns=headers)
    defense3_df['Team'] = defense3_df['Team'].str.replace('St.', 'State', regex=False)
else:
    Defense3_df = pd.DataFrame(data) 

defense3_df

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,Air Force,13,174,13.4,1239,95.3,1887,145.2,3126,240.5
1,Iowa,13,173,13.3,1317,101.3,1962,150.9,3279,252.2
2,Illinois,13,166,12.8,1297,99.8,2066,158.9,3363,258.7
3,James Madison,11,230,20.9,882,80.2,2112,192.0,2994,272.2
4,Iowa State,12,243,20.2,1288,107.3,2000,166.7,3288,274.0
...,...,...,...,...,...,...,...,...,...,...
126,Louisiana Tech,12,455,37.9,2918,243.2,2609,217.4,5527,460.6
127,Charlotte,12,473,39.4,2469,205.8,3235,269.6,5704,475.3
128,Georgia S.,13,411,31.6,3002,230.9,3235,248.8,6237,479.8
129,Colorado,12,534,44.5,2941,245.1,3109,259.1,6050,504.2


In [53]:
defense3_df = defense3_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
defense3_df.head()

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,Air Force,13.4,95.3,145.2,240.5
1,Iowa,13.3,101.3,150.9,252.2
2,Illinois,12.8,99.8,158.9,258.7
3,James Madison,20.9,80.2,192.0,272.2
4,Iowa State,20.2,107.3,166.7,274.0


In [54]:
defense3_df = defense3_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS ALLOWED/GAME",
    "RYds/G": "RUSH YDS ALLOWED/GAME",
    "PYds/G": "PASS YDS ALLOWED/GAME",
    "Yds/G": "YARDS ALLOWED/GAME"
})
defense3_df

,TEAM,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME
0,Air Force,13.4,95.3,145.2,240.5
1,Iowa,13.3,101.3,150.9,252.2
2,Illinois,12.8,99.8,158.9,258.7
3,James Madison,20.9,80.2,192.0,272.2
4,Iowa State,20.2,107.3,166.7,274.0
...,...,...,...,...,...
126,Louisiana Tech,37.9,243.2,217.4,460.6
127,Charlotte,39.4,205.8,269.6,475.3
128,Georgia S.,31.6,230.9,248.8,479.8
129,Colorado,44.5,245.1,259.1,504.2


In [55]:
defense3_df["POINTS ALLOWED/GAME"] = pd.to_numeric(defense3_df["POINTS ALLOWED/GAME"])
defense3_df["RUSH YDS ALLOWED/GAME"] = pd.to_numeric(defense3_df["RUSH YDS ALLOWED/GAME"])
defense3_df["PASS YDS ALLOWED/GAME"] = pd.to_numeric(defense3_df["PASS YDS ALLOWED/GAME"])
defense3_df["YARDS ALLOWED/GAME"] = pd.to_numeric(defense3_df["YARDS ALLOWED/GAME"])
defense3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TEAM                   131 non-null    object 
 1   POINTS ALLOWED/GAME    131 non-null    float64
 2   RUSH YDS ALLOWED/GAME  131 non-null    float64
 3   PASS YDS ALLOWED/GAME  131 non-null    float64
 4   YARDS ALLOWED/GAME     131 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.2+ KB


# 2022 Offense

In [57]:
service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Offensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=O&cat=T&yr=2022&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)
            
except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    offense3_df = pd.DataFrame(data, columns=headers)
    offense3_df['Team'] = offense3_df['Team'].str.replace('St.', 'State', regex=False)
else:
    offense3_df = pd.DataFrame(data) 

offense3_df         

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,Tennessee,13,599,46.1,2593,199.5,4096,315.1,6689,514.5
1,Washington,13,516,39.7,1898,146.0,4753,365.6,6651,511.6
2,Oregon,13,504,38.8,2805,215.8,3678,282.9,6483,498.7
3,Georgia,15,616,41.1,3080,205.3,4370,291.3,7450,496.7
4,UCLA,13,510,39.2,3098,238.3,3334,256.5,6432,494.8
...,...,...,...,...,...,...,...,...,...,...
126,Colorado,12,185,15.4,1300,108.3,1911,159.2,3211,267.6
127,Colorado State,12,158,13.2,1068,89.0,1960,163.3,3028,252.3
128,Massachusetts,12,150,12.5,1786,148.8,1218,101.5,3004,250.3
129,Iowa,13,230,17.7,1229,94.5,1755,135.0,2984,229.5


In [58]:
offense3_df = offense3_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
offense3_df

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,Tennessee,46.1,199.5,315.1,514.5
1,Washington,39.7,146.0,365.6,511.6
2,Oregon,38.8,215.8,282.9,498.7
3,Georgia,41.1,205.3,291.3,496.7
4,UCLA,39.2,238.3,256.5,494.8
...,...,...,...,...,...
126,Colorado,15.4,108.3,159.2,267.6
127,Colorado State,13.2,89.0,163.3,252.3
128,Massachusetts,12.5,148.8,101.5,250.3
129,Iowa,17.7,94.5,135.0,229.5


In [59]:
offense3_df = offense3_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS/GAME",
    "RYds/G": "RUSH YDS/GAME",
    "PYds/G": "PASS YDS/GAME",
    "Yds/G": "YARDS/GAME"
})
offense3_df.head(5)

,TEAM,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME
0,Tennessee,46.1,199.5,315.1,514.5
1,Washington,39.7,146.0,365.6,511.6
2,Oregon,38.8,215.8,282.9,498.7
3,Georgia,41.1,205.3,291.3,496.7
4,UCLA,39.2,238.3,256.5,494.8


In [60]:
offense3_df["POINTS/GAME"] = pd.to_numeric(offense3_df["POINTS/GAME"])
offense3_df["RUSH YDS/GAME"] = pd.to_numeric(offense3_df["RUSH YDS/GAME"])
offense3_df["PASS YDS/GAME"] = pd.to_numeric(offense3_df["PASS YDS/GAME"])
offense3_df["YARDS/GAME"] = pd.to_numeric(offense3_df["YARDS/GAME"])
offense3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TEAM           131 non-null    object 
 1   POINTS/GAME    131 non-null    float64
 2   RUSH YDS/GAME  131 non-null    float64
 3   PASS YDS/GAME  131 non-null    float64
 4   YARDS/GAME     131 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.2+ KB


In [61]:
defense3_df['TEAM'] = defense3_df['TEAM'].str.replace('S. Carolina', 'South Carolina', regex=False)
offense3_df['TEAM'] = offense3_df['TEAM'].str.replace('S. Carolina', 'South Carolina', regex=False)
defense3_df['TEAM'] = defense3_df['TEAM'].str.replace('Mississippi State', 'Mississippi State', regex=False)
offense3_df['TEAM'] = offense3_df['TEAM'].str.replace('Mississippi State', 'Mississippi State', regex=False)

In [62]:
# Merge rank_df with defense_df on 'TEAM'
merged3_df = rank_df3.merge(defense3_df, on="TEAM", how="left")

# Merge the result with offense_df on 'TEAM'
merged3_df = merged3_df.merge(offense3_df, on="TEAM", how="left")
merged3_df['Season'] = 2022
# Display the final merged DataFrame
merged3_df

,RANK,TEAM,RECORD,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME,Season
0,1,Georgia,15-0,14.3,77.0,204.5,281.5,41.1,205.3,291.3,496.7,2022
1,2,TCU,13-2,29.0,158.9,236.9,395.7,38.8,193.3,246.5,439.8,2022
2,3,Michigan,13-1,16.1,97.9,176.4,274.4,40.4,238.9,209.9,448.8,2022
3,4,Ohio State,11-2,21.0,121.1,184.5,305.6,44.2,192.4,290.8,483.2,2022
4,5,Alabama,11-2,18.2,130.4,170.2,300.5,41.1,195.7,269.6,465.3,2022
5,6,Tennessee,11-2,22.8,116.1,273.2,389.2,46.1,199.5,315.1,514.5,2022
6,7,Penn State,11-2,18.2,111.2,190.4,301.5,35.8,181.1,243.0,424.1,2022
7,8,Washington,11-2,25.8,121.2,232.1,353.3,39.7,146.0,365.6,511.6,2022
8,9,Tulane,12-2,22.2,152.3,197.0,349.3,36.0,204.8,226.0,430.8,2022
9,10,Utah,10-4,21.4,111.4,201.9,313.3,38.6,217.7,241.5,459.2,2022


# 2021 Season

In [64]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the ESPN rankings page
url = "https://www.espn.com/college-football/rankings/_/poll/1/week/1/year/2021/seasontype/3"
browser.get(url)

time.sleep(random.uniform(2, 5))

try:
    table = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
    rows = table.find_elements(By.TAG_NAME, "tr")

    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = [col.text.strip() for col in cols]
        if cols_text:  # Avoid empty rows
            data.append(cols_text)

except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

rank_df4 = pd.DataFrame(data, columns=headers)
rank_df4.head()

,RK,TEAM,REC,PTS,TREND,LAST WEEK,NEXT WEEK
0,1,Georgia\n(61),14-1,1525,-,vs #2AlabamaW33-18,-
1,2,Alabama,13-2,1464,-,vs #1GeorgiaL33-18,-
2,3,Michigan,12-2,1361,-,vs #1GeorgiaL34-11,-
3,4,Cincinnati,13-1,1349,-,vs #2AlabamaL27-6,-
4,5,Baylor,12-2,1259,-,vs #11Ole MissW21-7,-


In [65]:
rank_df4 = rank_df4[["RK", "TEAM", "REC"]] 
rank_df4

,RK,TEAM,REC
0,1,Georgia\n(61),14-1
1,2,Alabama,13-2
2,3,Michigan,12-2
3,4,Cincinnati,13-1
4,5,Baylor,12-2
5,6,Ohio State,11-2
6,7,Oklahoma State,12-2
7,8,Notre Dame,11-2
8,9,Michigan State,11-2
9,10,Oklahoma,11-2


In [66]:
rank_df4["RK"] = pd.to_numeric(rank_df4["RK"])
rank_df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RK      25 non-null     int64 
 1   TEAM    25 non-null     object
 2   REC     25 non-null     object
dtypes: int64(1), object(2)
memory usage: 732.0+ bytes


In [67]:
import re
rank_df4["TEAM"] = rank_df4["TEAM"].str.replace(r"[^a-zA-Z\s]", "", regex=True).str.strip()
rank_df4.head()

,RK,TEAM,REC
0,1,Georgia,14-1
1,2,Alabama,13-2
2,3,Michigan,12-2
3,4,Cincinnati,13-1
4,5,Baylor,12-2


In [68]:
rank_df4 = rank_df4.rename(columns = {
    'RK' : 'RANK',
    'REC' : 'RECORD'
})
rank_df4.head()

,RANK,TEAM,RECORD
0,1,Georgia,14-1
1,2,Alabama,13-2
2,3,Michigan,12-2
3,4,Cincinnati,13-1
4,5,Baylor,12-2


# 2021 Defense

In [70]:
service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Defensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=D&cat=T&yr=2021&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)
except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    defense4_df = pd.DataFrame(data, columns=headers)
    defense4_df['Team'] = defense4_df['Team'].str.replace('St.', 'State', regex=False)
else:
    Defense4_df = pd.DataFrame(data) 

defense4_df

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,Wisconsin,13,210,16.2,842,64.8,2007,154.4,2849,219.2
1,Georgia,15,153,10.2,1180,78.7,2476,165.1,3656,243.7
2,Minnesota,13,225,17.3,1268,97.5,2180,167.7,3448,265.2
3,Oklahoma State,14,253,18.1,1227,87.6,2585,184.6,3812,272.3
4,Air Force,13,257,19.8,1370,105.4,2210,170.0,3580,275.4
...,...,...,...,...,...,...,...,...,...,...
125,Kansas,12,506,42.2,3005,250.4,2728,227.3,5733,477.8
126,Massachusetts,12,517,43.1,2820,235.0,2922,243.5,5742,478.5
127,FIU,12,476,39.7,2772,231.0,3070,255.8,5842,486.8
128,Arkansas State,12,463,38.6,3133,261.1,2821,235.1,5954,496.2


In [71]:
defense4_df = defense4_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
defense4_df.head()

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,Wisconsin,16.2,64.8,154.4,219.2
1,Georgia,10.2,78.7,165.1,243.7
2,Minnesota,17.3,97.5,167.7,265.2
3,Oklahoma State,18.1,87.6,184.6,272.3
4,Air Force,19.8,105.4,170.0,275.4


In [72]:
defense4_df = defense4_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS ALLOWED/GAME",
    "RYds/G": "RUSH YDS ALLOWED/GAME",
    "PYds/G": "PASS YDS ALLOWED/GAME",
    "Yds/G": "YARDS ALLOWED/GAME"
})
defense4_df

,TEAM,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME
0,Wisconsin,16.2,64.8,154.4,219.2
1,Georgia,10.2,78.7,165.1,243.7
2,Minnesota,17.3,97.5,167.7,265.2
3,Oklahoma State,18.1,87.6,184.6,272.3
4,Air Force,19.8,105.4,170.0,275.4
...,...,...,...,...,...
125,Kansas,42.2,250.4,227.3,477.8
126,Massachusetts,43.1,235.0,243.5,478.5
127,FIU,39.7,231.0,255.8,486.8
128,Arkansas State,38.6,261.1,235.1,496.2


In [73]:
defense4_df["POINTS ALLOWED/GAME"] = pd.to_numeric(defense4_df["POINTS ALLOWED/GAME"])
defense4_df["RUSH YDS ALLOWED/GAME"] = pd.to_numeric(defense4_df["RUSH YDS ALLOWED/GAME"])
defense4_df["PASS YDS ALLOWED/GAME"] = pd.to_numeric(defense4_df["PASS YDS ALLOWED/GAME"])
defense4_df["YARDS ALLOWED/GAME"] = pd.to_numeric(defense4_df["YARDS ALLOWED/GAME"])
defense4_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TEAM                   130 non-null    object 
 1   POINTS ALLOWED/GAME    130 non-null    float64
 2   RUSH YDS ALLOWED/GAME  130 non-null    float64
 3   PASS YDS ALLOWED/GAME  130 non-null    float64
 4   YARDS ALLOWED/GAME     130 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.2+ KB


# 2021 Offense

In [75]:
service = Service()
browser = webdriver.Chrome(service=service)

# URL of the Offensive Stats Page
url = "https://www.footballdb.com/college-football/stats/teamstat.html?group=O&cat=T&yr=2021&lg=FBS"
browser.get(url)

try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    headers = [th.text.strip() for th in browser.find_elements(By.XPATH, "//table//thead//tr/th")]

    rows = browser.find_elements(By.XPATH, "//table//tbody//tr")
    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        cols_text = []

        for i, col in enumerate(cols):
            if headers[i] == 'Team':  
                try:
                    team_name_element = col.find_elements(By.XPATH, ".//a")[1]
                    team_name = team_name_element.text.strip()
                    # Replace "St." with "State" in the team name
                    team_name = team_name.replace("St.", "State")
                except IndexError:
                    team_name = col.text.strip().replace("St.", "State")
                except Exception as e:
                    print(f"Error extracting team name: {e}")
                    team_name = ""
                cols_text.append(team_name)
            else:
                cols_text.append(col.text.strip())

        if cols_text:  
            data.append(cols_text)
except Exception as e:
    print(f"Error extracting data: {e}")

browser.quit()

if headers and data:
    offense4_df = pd.DataFrame(data, columns=headers)
    offense4_df['Team'] = offense4_df['Team'].str.replace('St.', 'State', regex=False)
else:
    offense4_df = pd.DataFrame(data) 

offense4_df                    

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,Ohio State,13,594,45.7,2344,180.3,4819,370.7,7163,551.0
1,W. Kentucky,14,619,44.2,1450,103.6,5935,423.9,7385,527.5
2,Virginia,12,415,34.6,1467,122.2,4482,373.5,5949,495.8
3,Coastal Carolina,13,532,40.9,2974,228.8,3336,256.6,6310,485.4
4,Kent State,14,462,33.0,3482,248.7,3196,228.3,6678,477.0
...,...,...,...,...,...,...,...,...,...,...
125,Navy,12,241,20.1,2705,225.4,516,43.0,3221,268.4
126,Connecticut,12,187,15.6,1215,101.2,1708,142.3,2923,243.6
127,Colorado,12,225,18.8,1515,126.2,1374,114.5,2889,240.8
128,S. Mississippi,12,212,17.7,1449,120.8,1397,116.4,2846,237.2


In [76]:
offense4_df = offense4_df[["Team", "Pts/G", "RYds/G", "PYds/G", "Yds/G"]] 
offense4_df

,Team,Pts/G,RYds/G,PYds/G,Yds/G
0,Ohio State,45.7,180.3,370.7,551.0
1,W. Kentucky,44.2,103.6,423.9,527.5
2,Virginia,34.6,122.2,373.5,495.8
3,Coastal Carolina,40.9,228.8,256.6,485.4
4,Kent State,33.0,248.7,228.3,477.0
...,...,...,...,...,...
125,Navy,20.1,225.4,43.0,268.4
126,Connecticut,15.6,101.2,142.3,243.6
127,Colorado,18.8,126.2,114.5,240.8
128,S. Mississippi,17.7,120.8,116.4,237.2


In [77]:
offense4_df = offense4_df.rename(columns={
    "Team": "TEAM",
    "Pts/G": "POINTS/GAME",
    "RYds/G": "RUSH YDS/GAME",
    "PYds/G": "PASS YDS/GAME",
    "Yds/G": "YARDS/GAME"
})
offense4_df.head(5)

,TEAM,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME
0,Ohio State,45.7,180.3,370.7,551.0
1,W. Kentucky,44.2,103.6,423.9,527.5
2,Virginia,34.6,122.2,373.5,495.8
3,Coastal Carolina,40.9,228.8,256.6,485.4
4,Kent State,33.0,248.7,228.3,477.0


In [78]:
offense4_df["POINTS/GAME"] = pd.to_numeric(offense4_df["POINTS/GAME"])
offense4_df["RUSH YDS/GAME"] = pd.to_numeric(offense4_df["RUSH YDS/GAME"])
offense4_df["PASS YDS/GAME"] = pd.to_numeric(offense4_df["PASS YDS/GAME"])
offense4_df["YARDS/GAME"] = pd.to_numeric(offense4_df["YARDS/GAME"])
offense4_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TEAM           130 non-null    object 
 1   POINTS/GAME    130 non-null    float64
 2   RUSH YDS/GAME  130 non-null    float64
 3   PASS YDS/GAME  130 non-null    float64
 4   YARDS/GAME     130 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.2+ KB


In [79]:
#Cleaning the data so the team names match with ranks_df
defense4_df['TEAM'] = defense4_df['TEAM'].str.replace('N. Carolina State', 'NC State', regex=False)
offense4_df['TEAM'] = offense4_df['TEAM'].str.replace('N. Carolina State', 'NC State', regex=False)
defense4_df['TEAM'] = defense4_df['TEAM'].str.replace('Mississippi', 'Ole Miss', regex=False)
offense4_df['TEAM'] = offense4_df['TEAM'].str.replace('Mississippi', 'Ole Miss', regex=False)

In [80]:
# Merge rank_df with defense_df on 'TEAM'
merged4_df = rank_df4.merge(defense4_df, on="TEAM", how="left")

# Merge the result with offense_df on 'TEAM'
merged4_df = merged4_df.merge(offense4_df, on="TEAM", how="left")
merged4_df['Season'] = 2021
# Display the final merged DataFrame
merged4_df

,RANK,TEAM,RECORD,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME,Season
0,1,Georgia,14-1,10.2,78.7,165.1,243.7,38.6,190.9,243.2,434.1,2021
1,2,Alabama,13-2,20.1,86.1,193.1,279.1,39.9,150.0,319.1,469.1,2021
2,3,Michigan,12-2,17.4,126.4,189.4,315.9,35.8,214.1,220.7,434.9,2021
3,4,Cincinnati,13-1,16.9,149.1,152.5,301.6,36.9,171.9,229.1,401.1,2021
4,5,Baylor,12-2,18.3,118.0,206.4,324.4,31.6,219.3,193.9,413.2,2021
5,6,Ohio State,11-2,22.8,127.8,227.2,355.0,45.7,180.3,370.7,551.0,2021
6,7,Oklahoma State,12-2,18.1,87.6,184.6,272.3,31.1,188.1,224.5,412.6,2021
7,8,Notre Dame,11-2,19.7,135.6,202.2,337.8,35.2,143.8,266.2,410.0,2021
8,9,Michigan State,11-2,25.3,117.2,301.8,419.0,31.8,175.6,242.1,417.7,2021
9,10,Oklahoma,11-2,25.8,129.1,247.2,376.3,39.1,187.2,249.5,436.8,2021


## Final DF

In [101]:
final_df = pd.concat([merged_df, merged2_df, merged3_df, merged4_df], axis=0)
final_df['Season'] = pd.to_datetime(final_df['Season'], format='%Y')
final_df['Season'] = final_df['Season'].dt.year
final_df

,RANK,TEAM,RECORD,POINTS ALLOWED/GAME,RUSH YDS ALLOWED/GAME,PASS YDS ALLOWED/GAME,YARDS ALLOWED/GAME,POINTS/GAME,RUSH YDS/GAME,PASS YDS/GAME,YARDS/GAME,Season
0,1,Ohio State,14-2,12.9,87.6,144.1,231.8,35.7,166.4,256.4,422.8,2024
1,2,Notre Dame,14-2,15.5,138.1,152.8,290.9,36.1,200.9,189.5,390.4,2024
2,3,Oregon,13-1,19.4,130.1,171.1,301.2,34.9,157.9,268.5,426.4,2024
3,4,Texas,13-3,15.3,109.9,154.7,264.6,33.0,158.8,264.9,423.7,2024
4,5,Penn State,13-3,16.5,101.9,173.6,275.4,33.1,202.3,219.0,421.3,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
20,21,Arkansas,9-4,22.9,153.8,202.0,355.8,30.9,227.8,199.8,427.5,2021
21,22,Oregon,10-4,27.0,143.2,230.4,373.6,31.4,202.4,212.9,415.3,2021
22,23,Iowa,10-4,19.2,114.4,199.2,313.6,23.4,123.6,165.3,288.9,2021
23,24,Utah State,11-3,24.4,161.9,216.4,378.2,32.6,142.6,289.0,431.6,2021


In [103]:
final_df.to_csv("2021-2024stats.csv", index=False)